# Minerva Agent Workforce — Full Demo Notebook

This notebook demonstrates every agent in the Minerva system, triggers each one to do real work,
and shows how the **Conductor** orchestrates the whole workforce into a daily digest.

## Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                    CONDUCTOR (Orchestrator)                  │
│  Runs daily — delegates to sub-agents — compiles digest     │
└──────────┬────────┬────────┬────────┬────────┬─────────────┘
           │        │        │        │        │
    ┌──────▼──┐ ┌───▼────┐ ┌▼──────┐ ┌▼──────┐ ┌▼──────────┐
    │Engineer │ │Sentinel│ │Advocate│ │Telemetry│ │Productizer│
    │Gap Ana- │ │QA Gate-│ │GitHub │ │Usage   │ │Sales Kits │
    │lysis    │ │keeper  │ │Triage │ │Metrics │ │Generator  │
    └─────────┘ └────────┘ └───────┘ └────────┘ └───────────┘
    ┌──────────┐ ┌─────────┐ ┌─────────────┐
    │Marketing │ │Support  │ │Presentation │
    │Social    │ │FAQ Bot  │ │PowerPoint   │
    │Drafts    │ │         │ │Generator    │
    └──────────┘ └─────────┘ └─────────────┘
```

**All agents run in `dry_run=True` mode** — they report what they would do without side effects.

---
## 0. Setup — Add Project Root to Path

In [ ]:
import sys
import os
from pathlib import Path

# In Jupyter, __file__ doesn't exist. Use cwd-based detection.
# This notebook lives in notebooks/ → project root is the parent.
_notebook_dir = Path(os.path.abspath(""))
PROJECT_ROOT = _notebook_dir.parent if _notebook_dir.name == "notebooks" else _notebook_dir

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# Also load .env if present
try:
    from dotenv import load_dotenv
    load_dotenv(PROJECT_ROOT / ".env", override=False)
    print("✅ .env loaded")
except ImportError:
    print("ℹ️  python-dotenv not installed — .env not loaded")

# Verify LLM availability
from llm_manager.llm_interface import is_llm_available, get_active_model_name
llm_ok = is_llm_available()
print(f"{'✅' if llm_ok else '⚠️ '} LLM available: {llm_ok} — active model: {get_active_model_name()}")
print(f"📁 Project root: {PROJECT_ROOT}")

---
## 1. Generate Demo Data

We create a synthetic **retail e-commerce** dataset that several agents will use.
This mimics a real upload without needing external data.

In [ ]:
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
import random

# Seed for reproducibility
np.random.seed(42)
random.seed(42)

# ── Synthetic Retail Dataset ──────────────────────────────────────────────────
n = 500
categories = ["Electronics", "Clothing", "Home & Garden", "Sports", "Books"]
regions = ["North", "South", "East", "West", "Central"]
channels = ["Online", "In-Store", "Mobile App"]

dates = [datetime(2025, 1, 1) + timedelta(days=random.randint(0, 364)) for _ in range(n)]

df_retail = pd.DataFrame({
    "order_id":        range(10001, 10001 + n),
    "order_date":      [d.strftime("%Y-%m-%d") for d in dates],
    "customer_id":     np.random.randint(1000, 2000, n),
    "product_category": np.random.choice(categories, n, p=[0.30, 0.25, 0.20, 0.15, 0.10]),
    "revenue":         np.round(np.random.lognormal(4.5, 0.8, n), 2),
    "units_sold":      np.random.randint(1, 15, n),
    "discount_pct":    np.round(np.random.uniform(0, 0.35, n), 2),
    "region":          np.random.choice(regions, n),
    "channel":         np.random.choice(channels, n, p=[0.55, 0.30, 0.15]),
    "customer_rating": np.random.choice([1, 2, 3, 4, 5], n, p=[0.03, 0.07, 0.15, 0.35, 0.40]),
    "is_returned":     np.random.choice([0, 1], n, p=[0.88, 0.12]),
    "days_to_ship":    np.random.randint(1, 8, n),
})

# Save to User_Data (for Productizer)
user_data_dir = PROJECT_ROOT / "User_Data"
user_data_dir.mkdir(exist_ok=True)
demo_csv_path = user_data_dir / "demo_retail_sales.csv"
df_retail.to_csv(demo_csv_path, index=False)

print(f"✅ Created demo dataset: {demo_csv_path}")
print(f"   Shape: {df_retail.shape}")
print(f"   Columns: {list(df_retail.columns)}")
df_retail.head()

---
## 2. LLM Interface Demo

Before we trigger agents, let's see the LLM layer in action — analyzing data and suggesting visualizations.

In [ ]:
from llm_manager.llm_interface import analyze_data_with_llm, suggest_visualizations, get_llm_completion

print("═" * 60)
print("  LLM INTERFACE: Data Analysis")
print("═" * 60)

question = "Which product category drives the most revenue, and are there any return rate patterns I should worry about?"
print(f"\nQuestion: {question}\n")

if is_llm_available():
    analysis = analyze_data_with_llm(df_retail, question)
    print(analysis if analysis else "[No response — LLM may need configuration]")
else:
    # Show what the prompt looks like
    print("⚠️  No LLM configured. Here's a preview of what Minerva would send:")
    stats_summary = df_retail.describe().to_string()
    sample = df_retail.head(3).to_string()
    print(f"\nDataset: {df_retail.shape[0]} rows × {df_retail.shape[1]} cols")
    print(f"Key stats: Total revenue = ${df_retail['revenue'].sum():,.2f}")
    by_cat = df_retail.groupby('product_category')['revenue'].sum().sort_values(ascending=False)
    print(f"Top category by revenue: {by_cat.index[0]} (${by_cat.iloc[0]:,.2f})")
    return_rates = df_retail.groupby('product_category')['is_returned'].mean()
    print(f"Highest return rate: {return_rates.idxmax()} ({return_rates.max():.1%})")

In [ ]:
print("═" * 60)
print("  LLM INTERFACE: Visualization Suggestions")
print("═" * 60)

if is_llm_available():
    suggestions = suggest_visualizations(df_retail)
    print(f"\nSuggested {len(suggestions)} visualizations:\n")
    for i, s in enumerate(suggestions, 1):
        print(f"{i}. [{s.get('type','?').upper()}] {s.get('title','Untitled')}")
        print(f"   X: {s.get('x','?')}  |  Y: {s.get('y','?')}")
        if s.get('reason'):
            print(f"   Why: {s.get('reason')}")
        print()
else:
    # Deterministic fallback suggestions
    suggestions = [
        {"type": "bar", "x": "product_category", "y": "revenue", "title": "Revenue by Category"},
        {"type": "line", "x": "order_date", "y": "revenue", "title": "Revenue Over Time"},
        {"type": "scatter", "x": "discount_pct", "y": "revenue", "title": "Discount vs Revenue"},
    ]
    print("\n[No LLM — using heuristic suggestions]")
    for i, s in enumerate(suggestions, 1):
        print(f"{i}. [{s['type'].upper()}] {s['title']} — X:{s['x']} / Y:{s['y']}")

In [ ]:
# Render the suggested charts
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Chart 1: Revenue by category
cat_revenue = df_retail.groupby('product_category')['revenue'].sum().reset_index().sort_values('revenue', ascending=True)
fig1 = px.bar(cat_revenue, x='revenue', y='product_category', orientation='h',
              title='Total Revenue by Product Category',
              color='revenue', color_continuous_scale='Viridis',
              labels={'revenue': 'Total Revenue ($)', 'product_category': 'Category'})
fig1.show()

# Chart 2: Return rate + avg rating by category
cat_stats = df_retail.groupby('product_category').agg(
    return_rate=('is_returned', 'mean'),
    avg_rating=('customer_rating', 'mean'),
    orders=('order_id', 'count')
).reset_index()

fig2 = make_subplots(rows=1, cols=2, subplot_titles=['Return Rate by Category', 'Avg Rating by Category'])
fig2.add_trace(go.Bar(x=cat_stats['product_category'], y=cat_stats['return_rate'],
                      name='Return Rate', marker_color='#ef5350'), row=1, col=1)
fig2.add_trace(go.Bar(x=cat_stats['product_category'], y=cat_stats['avg_rating'],
                      name='Avg Rating', marker_color='#42a5f5'), row=1, col=2)
fig2.update_layout(title='Quality Metrics by Category', showlegend=False)
fig2.show()

# Chart 3: Revenue by channel over time (monthly)
df_retail['month'] = pd.to_datetime(df_retail['order_date']).dt.to_period('M').astype(str)
monthly_channel = df_retail.groupby(['month', 'channel'])['revenue'].sum().reset_index()
fig3 = px.line(monthly_channel, x='month', y='revenue', color='channel',
               title='Monthly Revenue by Sales Channel',
               labels={'revenue': 'Revenue ($)', 'month': 'Month'})
fig3.show()

print("✅ Charts rendered")

---
## 3. Agent Setup — Load All Agents

We instantiate every agent in dry-run mode.

In [ ]:
from agents.base import AgentConfig, AgentResult, Priority
from agents.conductor import ConductorAgent
from agents.engineer import EngineerAgent
from agents.sentinel import SentinelAgent
from agents.advocate import AdvocateAgent
from agents.telemetry import TelemetryAgent
from agents.productizer import ProductizerAgent
from agents.marketing import MarketingAgent
from agents.support import SupportAgent
from agents.presentation import PresentationAgent

def make_config(name, enabled=True, dry_run=True, extra=None):
    return AgentConfig(
        name=name,
        enabled=enabled,
        dry_run=dry_run,
        llm_model="claude-3-5-sonnet",
        extra=extra or {}
    )

# Instantiate all agents
agents = {
    "engineer":     EngineerAgent(config=make_config("engineer")),
    "sentinel":     SentinelAgent(config=make_config("sentinel", extra={"min_confidence_score": 7})),
    "advocate":     AdvocateAgent(config=make_config("advocate", extra={"stale_days": 14})),
    "telemetry":    TelemetryAgent(config=make_config("telemetry")),
    "productizer":  ProductizerAgent(config=make_config("productizer")),
    "marketing":    MarketingAgent(config=make_config("marketing")),
    "support":      SupportAgent(config=make_config("support")),
    "presentation": PresentationAgent(config=make_config("presentation")),
}

print("Minerva Agent Workforce")
print("═" * 55)
emoji_map = {
    "engineer": "🔧", "sentinel": "🛡️", "advocate": "🤝",
    "telemetry": "📊", "productizer": "💼", "marketing": "📣",
    "support": "💬", "presentation": "📑",
}
role_map = {
    "engineer": "Codebase gap analysis & improvement recommender",
    "sentinel": "QA gatekeeper — tests, lint, confidence scoring",
    "advocate": "GitHub issue triage & community management",
    "telemetry": "Usage analytics & health monitoring",
    "productizer": "Vertical sales kit generator",
    "marketing": "Social media draft generator from git log",
    "support": "FAQ-backed intelligent issue responder",
    "presentation": "PowerPoint generator from sales kits",
}
for name, agent in agents.items():
    e = emoji_map.get(name, "•")
    mode = "[dry-run]" if agent.is_dry_run else "[LIVE]"
    print(f"  {e} {name:14s} {mode}  {role_map.get(name, '')}")
print()
print(f"  All {len(agents)} agents ready.")

---
## 4. Engineer Agent — Codebase Gap Analysis

The Engineer reads subsystem source files and scores their maturity against best-in-class standards.
It writes the report to `knowledge_base/product/vision_gap_analysis.md`.

In [ ]:
import time

print("═" * 60)
print("  ENGINEER AGENT — Gap Analysis")
print("═" * 60)

start = time.time()
engineer_result = agents["engineer"].run()
elapsed = time.time() - start

status = "✅" if engineer_result.success else "❌"
print(f"\n{status} Status: {'Success' if engineer_result.success else 'Failed'}")
print(f"⏱️  Duration: {elapsed:.1f}s")
print(f"📝 Summary: {engineer_result.summary}")

if engineer_result.metrics:
    print("\n📊 Metrics:")
    for k, v in engineer_result.metrics.items():
        if isinstance(v, float):
            print(f"   {k}: {v:.2f}")
        else:
            print(f"   {k}: {v}")

print("\n🔧 Actions taken:")
for action in engineer_result.actions_taken:
    print(f"   → {action}")

if engineer_result.escalations:
    print("\n⚠️  Escalations:")
    for esc in engineer_result.escalations:
        emoji = {"urgent": "🔴", "review": "🟡", "fyi": "🟢", "metric": "📊"}.get(esc.priority.value, "•")
        print(f"   {emoji} [{esc.priority.value.upper()}] {esc.title}")
        print(f"      {esc.detail[:200]}")

if engineer_result.error:
    print(f"\n⚠️  Error: {engineer_result.error}")

In [ ]:
# Show the generated gap analysis report
from pathlib import Path

gap_report_path = PROJECT_ROOT / "agents" / "knowledge_base" / "product" / "vision_gap_analysis.md"
if gap_report_path.exists():
    from IPython.display import Markdown, display
    content = gap_report_path.read_text(encoding="utf-8")
    print(f"📄 Gap analysis report ({len(content)} chars):")
    display(Markdown(content))
else:
    print("(Gap analysis report not yet written — LLM may be unavailable)")

In [ ]:
# Visualize subsystem maturity scores
import plotly.express as px

# Engineer's SUBSYSTEMS with their key files
subsystems = {
    "cascade_planner": "orchestration/cascade_planner.py",
    "plan_learner": "orchestration/plan_learner.py",
    "mcp_server": "mcp_server/server.py",
    "api_discovery": "mcp_server/discovery_agent.py",
    "dynamic_connectors": "mcp_server/dynamic_connector.py",
    "model_training": "modeling/model_training.py",
    "smart_charts": "visualization/smart_charts.py",
    "nl_query": "ui/nl_query.py",
    "data_fabric": "ui/data_fabric.py",
    "dashboard": "ui/dashboard.py",
}

# Count lines as a quick proxy for maturity
maturity_data = []
for name, filepath in subsystems.items():
    full_path = PROJECT_ROOT / filepath
    if full_path.exists():
        lines = len(full_path.read_text(encoding="utf-8", errors="replace").splitlines())
        # Simple heuristic: <50=prototype, <200=functional, <500=production, 500+=best-in-class
        if lines < 50:   score, maturity = 1, "Prototype"
        elif lines < 200: score, maturity = 2, "Functional"
        elif lines < 500: score, maturity = 3, "Production"
        else:            score, maturity = 4, "Best-in-Class"
    else:
        lines, score, maturity = 0, 0, "Missing"
    
    maturity_data.append({
        "subsystem": name, "lines": lines, "score": score, "maturity": maturity
    })

df_maturity = pd.DataFrame(maturity_data).sort_values("score")

color_map = {
    "Missing": "#ef5350", "Prototype": "#ff7043",
    "Functional": "#ffa726", "Production": "#26a69a", "Best-in-Class": "#42a5f5"
}

fig = px.bar(
    df_maturity, x="score", y="subsystem", orientation="h",
    color="maturity", color_discrete_map=color_map,
    title="Subsystem Maturity Scores (Engineer Agent Assessment)",
    labels={"score": "Maturity Score (0-4)", "subsystem": "Subsystem"},
    range_x=[0, 4.5],
    text="lines",
    hover_data=["lines", "maturity"]
)
fig.update_traces(texttemplate='%{text} lines', textposition='outside')
fig.update_layout(height=450)
fig.show()

avg = df_maturity["score"].mean()
print(f"\nAverage maturity: {avg:.2f}/4 ({['Below Prototype','Prototype','Functional','Production','Best-in-Class'][round(avg)]})") 

---
## 5. Sentinel Agent — QA Validation

The Sentinel runs the test suite and linter, calculates a confidence score (1-10),
and routes the result: ≥7 → human review, <7 → back to Engineer.

In [ ]:
print("═" * 60)
print("  SENTINEL AGENT — QA Validation")
print("═" * 60)

start = time.time()
sentinel_result = agents["sentinel"].run(changed_files=[])
elapsed = time.time() - start

status = "✅" if sentinel_result.success else "❌"
print(f"\n{status} Status: {'Success' if sentinel_result.success else 'Failed'}")
print(f"⏱️  Duration: {elapsed:.1f}s")
print(f"📝 Summary: {sentinel_result.summary}")

if sentinel_result.metrics:
    tests = sentinel_result.metrics.get("tests", {})
    lint = sentinel_result.metrics.get("lint", {})
    score = sentinel_result.metrics.get("confidence_score", 0)

    print("\n🧪 Test Results:")
    print(f"   Passed:  {tests.get('passed', 0)}")
    print(f"   Failed:  {tests.get('failed', 0)}")
    print(f"   Errors:  {tests.get('errors', 0)}")
    print(f"   Total:   {tests.get('total', 0)}")
    print(f"   All OK:  {tests.get('all_passed', False)}")

    print("\n🔍 Lint Results:")
    print(f"   Issues:  {lint.get('issues', '?')}")
    print(f"   Clean:   {lint.get('clean', False)}")

    bar = "█" * score + "░" * (10 - score)
    verdict = "✅ Ready for human review" if score >= 7 else "⚠️ Needs iteration"
    print(f"\n🎯 Confidence Score: {bar} {score}/10")
    print(f"   Verdict: {verdict}")

if sentinel_result.escalations:
    print("\n📋 Escalations:")
    for esc in sentinel_result.escalations:
        emoji = {"urgent": "🔴", "review": "🟡", "fyi": "🟢", "metric": "📊"}.get(esc.priority.value, "•")
        print(f"   {emoji} [{esc.priority.value.upper()}] {esc.title}")

In [ ]:
# Visualize the confidence scoring breakdown
import plotly.graph_objects as go

score = sentinel_result.metrics.get("confidence_score", 0) if sentinel_result.metrics else 0
tests = sentinel_result.metrics.get("tests", {"passed": 0, "failed": 0, "errors": 0, "total": 0}) if sentinel_result.metrics else {}
lint = sentinel_result.metrics.get("lint", {"issues": 0, "clean": True}) if sentinel_result.metrics else {}

fig = go.Figure(go.Indicator(
    mode="gauge+number",
    value=score,
    domain={'x': [0, 1], 'y': [0, 1]},
    title={'text': "Sentinel Confidence Score"},
    gauge={
        'axis': {'range': [0, 10]},
        'bar': {'color': "#42a5f5" if score >= 7 else "#ffa726" if score >= 4 else "#ef5350"},
        'steps': [
            {'range': [0, 4], 'color': "#ffebee"},
            {'range': [4, 7], 'color': "#fff3e0"},
            {'range': [7, 10], 'color': "#e8f5e9"}
        ],
        'threshold': {
            'line': {'color': "green", 'width': 4},
            'thickness': 0.75,
            'value': 7
        }
    }
))
fig.update_layout(height=350)
fig.show()

# Test result breakdown
total = tests.get('total', 0)
if total > 0:
    fig2 = go.Figure(data=[go.Pie(
        labels=['Passed', 'Failed', 'Errors'],
        values=[tests.get('passed', 0), tests.get('failed', 0), tests.get('errors', 0)],
        hole=0.4,
        marker_colors=['#66bb6a', '#ef5350', '#ffa726']
    )])
    fig2.update_layout(title=f'Test Results ({total} total)', height=350)
    fig2.show()
else:
    print("(No tests found in project — run 'pytest' to populate)")

---
## 6. Advocate Agent — GitHub Issue Triage

The Advocate scans GitHub issues, classifies them (bug/feature/question/docs/security),
auto-labels them, generates responses for questions, and flags stale issues.

In dry-run mode, it reports what it **would** do without making any API calls.

In [ ]:
print("═" * 60)
print("  ADVOCATE AGENT — GitHub Issue Triage")
print("═" * 60)

start = time.time()
advocate_result = agents["advocate"].run()
elapsed = time.time() - start

status = "✅" if advocate_result.success else "❌"
print(f"\n{status} Status: {'Success' if advocate_result.success else 'Failed'}")
print(f"⏱️  Duration: {elapsed:.1f}s")
print(f"📝 Summary: {advocate_result.summary}")

if advocate_result.metrics:
    print("\n📊 Metrics:")
    for k, v in advocate_result.metrics.items():
        print(f"   {k}: {v}")

print("\n🔧 Actions taken:")
for action in advocate_result.actions_taken:
    print(f"   → {action}")

if advocate_result.escalations:
    print("\n⚠️  Escalations:")
    for esc in advocate_result.escalations:
        emoji = {"urgent": "🔴", "review": "🟡", "fyi": "🟢", "metric": "📊"}.get(esc.priority.value, "•")
        print(f"   {emoji} [{esc.priority.value.upper()}] {esc.title}")

In [ ]:
# Demo: Show how the Advocate classifies issues
print("\n  Issue Classifier Demo")
print("─" * 60)

sample_issues = [
    {"title": "App crashes when uploading large CSV files",
     "body": "Getting a traceback: MemoryError when I upload files > 500MB"},
    {"title": "Feature request: add support for Parquet format",
     "body": "Would be great to support .parquet files in addition to CSV and Excel"},
    {"title": "How do I configure the local LLM?",
     "body": "I have a GGUF model downloaded but I'm not sure how to point Minerva to it"},
    {"title": "SQL injection vulnerability in nl_query endpoint",
     "body": "The nl_query endpoint doesn't sanitize inputs and is vulnerable to SQL injection"},
    {"title": "Documentation for API discovery agent",
     "body": "The README doesn't explain how the API discovery works, need examples"},
]

# Use the Advocate's classifier directly
for issue in sample_issues:
    issue_type = agents["advocate"]._classify_issue(issue["title"], issue["body"])
    type_info = {
        "bug": ("🐛", "Bug"),
        "feature": ("✨", "Feature Request"),
        "question": ("❓", "Question"),
        "docs": ("📚", "Documentation"),
        "security": ("🔒", "Security"),
    }.get(issue_type, ("•", issue_type))
    
    print(f"  {type_info[0]} [{type_info[1]:18s}] {issue['title']}")

---
## 7. Telemetry Agent — Usage Analytics

The Telemetry Agent aggregates LLM interaction metrics, agent health data, and upload counts.
It writes insights to the knowledge base for the Productizer to use.

In [ ]:
print("═" * 60)
print("  TELEMETRY AGENT — Usage Analytics")
print("═" * 60)

start = time.time()
telemetry_result = agents["telemetry"].run(days=7)
elapsed = time.time() - start

status = "✅" if telemetry_result.success else "❌"
print(f"\n{status} Status: {'Success' if telemetry_result.success else 'Failed'}")
print(f"⏱️  Duration: {elapsed:.1f}s")
print(f"📝 Summary: {telemetry_result.summary}")

if telemetry_result.metrics:
    print("\n📊 Metrics:")
    for k, v in telemetry_result.metrics.items():
        if isinstance(v, float):
            print(f"   {k}: {v:.2%}" if "rate" in k else f"   {k}: {v:.1f}")
        else:
            print(f"   {k}: {v}")

print("\n🔧 Actions taken:")
for action in telemetry_result.actions_taken:
    print(f"   → {action}")

if telemetry_result.escalations:
    print("\n⚠️  Escalations:")
    for esc in telemetry_result.escalations:
        emoji = {"urgent": "🔴", "review": "🟡", "fyi": "🟢", "metric": "📊"}.get(esc.priority.value, "•")
        print(f"   {emoji} [{esc.priority.value.upper()}] {esc.title}")

# Check if telemetry KB artifact was written
telemetry_kb = PROJECT_ROOT / "agents" / "knowledge_base" / "operations" / "telemetry_insights.md"
if telemetry_kb.exists():
    print(f"\n📁 KB artifact written: {telemetry_kb.name}")

In [ ]:
# Show agent health dashboard (from Telemetry's data)
import plotly.graph_objects as go
from pathlib import Path
import sqlite3

state_dir = PROJECT_ROOT / "agents" / "state"
known_agents = ["conductor", "engineer", "sentinel", "advocate",
                "productizer", "marketing", "support", "telemetry"]

health_data = []
for agent_name in known_agents:
    db_path = state_dir / f"{agent_name}.db"
    if db_path.exists():
        try:
            with sqlite3.connect(str(db_path)) as conn:
                row = conn.execute(
                    "SELECT COUNT(*), SUM(success), AVG(duration_seconds) FROM runs"
                ).fetchone()
                runs = row[0] or 0
                success = row[1] or 0
                avg_dur = row[2] or 0
                health_data.append({
                    "agent": agent_name, "runs": runs,
                    "success_rate": round(success / runs, 2) if runs else 0,
                    "avg_duration": round(avg_dur, 1), "db_exists": True
                })
        except Exception:
            health_data.append({"agent": agent_name, "runs": 0, "success_rate": 0,
                                "avg_duration": 0, "db_exists": True})
    else:
        health_data.append({"agent": agent_name, "runs": 0, "success_rate": 0,
                            "avg_duration": 0, "db_exists": False})

df_health = pd.DataFrame(health_data)

fig = go.Figure(data=[
    go.Bar(name='Total Runs', x=df_health['agent'], y=df_health['runs'],
           marker_color='#42a5f5'),
    go.Bar(name='Success Rate (%)', x=df_health['agent'],
           y=df_health['success_rate'] * 100, marker_color='#66bb6a'),
])
fig.update_layout(
    title='Agent Health Dashboard',
    barmode='group',
    xaxis_title='Agent',
    yaxis_title='Count / Percentage',
    height=400
)
fig.show()
print(df_health.to_string(index=False))

---
## 8. Productizer Agent — Vertical Sales Kit

The Productizer analyzes our demo retail dataset and generates a complete sales kit:
buyer persona, pain points, elevator pitch, demo script, and ROI metrics.

In [ ]:
print("═" * 60)
print("  PRODUCTIZER AGENT — Sales Kit Generator")
print("═" * 60)

start = time.time()
# Pass the demo CSV we created earlier
productizer_result = agents["productizer"].run(file=demo_csv_path)
elapsed = time.time() - start

status = "✅" if productizer_result.success else "❌"
print(f"\n{status} Status: {'Success' if productizer_result.success else 'Failed'}")
print(f"⏱️  Duration: {elapsed:.1f}s")
print(f"📝 Summary: {productizer_result.summary}")

if productizer_result.metrics:
    print("\n📊 Metrics:")
    for k, v in productizer_result.metrics.items():
        print(f"   {k}: {v}")

print("\n🔧 Actions taken:")
for action in productizer_result.actions_taken:
    print(f"   → {action}")

if productizer_result.error:
    print(f"\n⚠️  Error: {productizer_result.error}")

In [ ]:
# Show the generated sales kit
from IPython.display import Markdown, display

sales_kit_dir = PROJECT_ROOT / "agents" / "knowledge_base" / "product" / "sales_kits"
kit_files = sorted(sales_kit_dir.glob("*.md"), key=lambda p: p.stat().st_mtime, reverse=True)

if kit_files:
    latest_kit = kit_files[0]
    content = latest_kit.read_text(encoding="utf-8")
    print(f"📄 Latest Sales Kit: {latest_kit.name} ({len(content)} chars)")
    print("─" * 60)
    display(Markdown(content))
else:
    print("(No sales kit found — LLM may be unavailable)")
    print("\nExpected kit structure:")
    print("""
    # Minerva Sales Kit: Retail & E-Commerce
    ## 1. The Executive Pitch
    ## 2. Pain Points (Why they buy)
    ## 3. ROI & Business Impact
    ## 4. The Golden Demo Script
    """)

---
## 9. Marketing Agent — Social Media Drafts

The Marketing Agent reads recent git commits, pulls from the Productizer's knowledge base,
and drafts platform-specific posts for HN, Reddit, and Twitter/X.

In [ ]:
print("═" * 60)
print("  MARKETING AGENT — Social Media Drafts")
print("═" * 60)

start = time.time()
marketing_result = agents["marketing"].run(days=7)
elapsed = time.time() - start

status = "✅" if marketing_result.success else "❌"
print(f"\n{status} Status: {'Success' if marketing_result.success else 'Failed'}")
print(f"⏱️  Duration: {elapsed:.1f}s")
print(f"📝 Summary: {marketing_result.summary}")

if marketing_result.metrics:
    print("\n📊 Metrics:")
    for k, v in marketing_result.metrics.items():
        if k != 'saved_paths':
            print(f"   {k}: {v}")

print("\n🔧 Actions taken:")
for action in marketing_result.actions_taken:
    print(f"   → {action}")

if marketing_result.escalations:
    print(f"\n✅ {len(marketing_result.escalations)} draft(s) escalated for review:")
    for esc in marketing_result.escalations:
        print(f"   → {esc.title}")

In [ ]:
# Show the generated marketing drafts
marketing_dir = PROJECT_ROOT / "agents" / "digests" / "marketing"
marketing_files = sorted(marketing_dir.glob("*.md"), key=lambda p: p.stat().st_mtime, reverse=True) if marketing_dir.exists() else []

if marketing_files:
    print(f"📄 Generated {len(marketing_files)} draft(s):")
    for draft_file in marketing_files[:3]:
        platform = draft_file.stem.split("_", 1)[-1].replace("_", " ").title()
        content = draft_file.read_text(encoding="utf-8")
        print(f"\n{'─'*60}")
        print(f"  {platform}: {draft_file.name}")
        print(f"{'─'*60}")
        print(content[:600] + ("..." if len(content) > 600 else ""))
else:
    print("(No marketing drafts found — LLM may be unavailable or no commits in last 7 days)")

# Show recent git commits the agent used
import subprocess
try:
    output = subprocess.check_output(
        ["git", "log", "--since=7.days.ago", "--pretty=format:%h %s", "--no-merges"],
        cwd=str(PROJECT_ROOT), stderr=subprocess.DEVNULL, timeout=10
    ).decode("utf-8").strip()
    if output:
        print("\n📦 Recent commits used as source:")
        for line in output.splitlines()[:5]:
            print(f"   {line}")
except Exception:
    pass

---
## 10. Support Agent — FAQ-Backed Responder

The Support Agent handles incoming questions against a structured FAQ + vector-store lookup.
High-confidence matches get auto-responded; low-confidence ones escalate to human review.

In [ ]:
print("═" * 60)
print("  SUPPORT AGENT — FAQ-Backed Responder")
print("═" * 60)

support_questions = [
    "How do I upload a CSV file to Minerva?",
    "Can Minerva handle Excel files?",
    "What LLM models does Minerva support?",
    "My analysis keeps failing with a timeout error",
    "How do I export results to PDF?",
]

for question in support_questions:
    start = time.time()
    result = agents["support"].run(question=question)
    elapsed = time.time() - start
    
    print(f"\n❓ Q: {question}")
    
    if result.escalations:
        for esc in result.escalations:
            emoji = {"urgent": "🔴", "review": "🟡", "fyi": "🟢", "metric": "📊"}.get(esc.priority.value, "•")
            if esc.priority.value == "fyi":
                # Auto-responded
                response = esc.detail[:300]
                print(f"   🟢 Auto-responded: {response[:200]}..." if len(response) > 200 else f"   🟢 Auto-responded: {response}")
            else:
                print(f"   {emoji} Escalated for human review: {esc.title}")
    else:
        print(f"   ℹ️  {result.summary}")
    
    print(f"   ⏱️  {elapsed:.2f}s")

In [ ]:
# Add a few FAQ entries to populate the knowledge base for future questions
from agents.support import SupportAgent

faq_entries = [
    {
        "question": "How do I upload a CSV file?",
        "answer": "In the Streamlit dashboard, go to the 'Data Upload' tab and drag-and-drop your CSV. "
                 "Files up to 200MB are supported. Run `minerva dashboard` to start the UI."
    },
    {
        "question": "What LLM models does Minerva support?",
        "answer": "Minerva supports: Anthropic Claude (cloud), OpenAI GPT models (cloud), "
                 "local GGUF models via llama.cpp, and Ollama models. Configure via the LLM Settings tab."
    },
    {
        "question": "How do I run Minerva in offline mode?",
        "answer": "Download a GGUF model file and place it in adm/llm_backends/local_model/. "
                 "Set ENABLE_LOCAL_LLM=1 in your .env file. Minerva will auto-detect and use it."
    },
    {
        "question": "How do I export results to PDF?",
        "answer": "Use the 'Export' button in the dashboard, or run `minerva analyze data.csv --export pdf`. "
                 "Reports are saved to the reports/ directory."
    }
]

for entry in faq_entries:
    agents["support"].add_faq_entry(entry["question"], entry["answer"])

print(f"✅ Added {len(faq_entries)} FAQ entries")
print(f"   FAQ size: {len(agents['support']._faq)} entries")

# Re-test with a question that now matches the FAQ
result = agents["support"].run(question="How do I upload files to Minerva?")
if result.escalations:
    esc = result.escalations[0]
    print(f"\n❓ Q: How do I upload files to Minerva?")
    print(f"   {'🟢 Auto-responded' if esc.priority.value == 'fyi' else '🟡 Escalated'}: {esc.detail[:200]}")

---
## 11. Presentation Agent — PowerPoint Generator

The Presentation Agent takes the latest Markdown Sales Kit and converts it
into a polished `.pptx` file ready for client meetings.

In [ ]:
print("═" * 60)
print("  PRESENTATION AGENT — PowerPoint Generator")
print("═" * 60)

start = time.time()
presentation_result = agents["presentation"].run()
elapsed = time.time() - start

status = "✅" if presentation_result.success else "❌"
print(f"\n{status} Status: {'Success' if presentation_result.success else 'Failed'}")
print(f"⏱️  Duration: {elapsed:.1f}s")
print(f"📝 Summary: {presentation_result.summary}")

if presentation_result.metrics:
    print("\n📊 Metrics:")
    for k, v in presentation_result.metrics.items():
        print(f"   {k}: {v}")

print("\n🔧 Actions taken:")
for action in presentation_result.actions_taken:
    print(f"   → {action}")

if presentation_result.error:
    print(f"\n⚠️  Error: {presentation_result.error}")

# Check if .pptx was created
pptx_dir = PROJECT_ROOT / "reports" / "presentations"
if pptx_dir.exists():
    pptx_files = sorted(pptx_dir.glob("*.pptx"), key=lambda p: p.stat().st_mtime, reverse=True)
    if pptx_files:
        print(f"\n📊 Presentation saved: {pptx_files[0].name}")
        print(f"   Size: {pptx_files[0].stat().st_size / 1024:.1f} KB")

---
## 12. Conductor Agent — Full Orchestration

The Conductor is the daily orchestrator. It:
1. Scans GitHub for repo stats
2. Runs all enabled sub-agents
3. Aggregates escalations by priority (🔴 Urgent / 🟡 Review / 🟢 FYI / 📊 Metric)
4. Compiles and saves the daily digest

In [ ]:
print("═" * 60)
print("  CONDUCTOR AGENT — Full Orchestration")
print("═" * 60)

conductor = ConductorAgent(config=make_config("conductor"))

start = time.time()
conductor_result = conductor.run()
elapsed = time.time() - start

status = "✅" if conductor_result.success else "❌"
print(f"\n{status} Status: {'Success' if conductor_result.success else 'Failed'}")
print(f"⏱️  Duration: {elapsed:.1f}s")
print(f"📝 Summary: {conductor_result.summary}")

if conductor_result.metrics:
    print("\n📊 GitHub Metrics:")
    for k, v in conductor_result.metrics.items():
        print(f"   {k}: {v}")

print("\n🔧 Conductor actions:")
for action in conductor_result.actions_taken:
    print(f"   → {action}")

# Tally escalations by priority
from collections import Counter
priority_counts = Counter(esc.priority.value for esc in conductor_result.escalations)

print(f"\n📋 Escalation Summary ({len(conductor_result.escalations)} total):")
print(f"   🔴 Urgent:  {priority_counts.get('urgent', 0)}")
print(f"   🟡 Review:  {priority_counts.get('review', 0)}")
print(f"   🟢 FYI:     {priority_counts.get('fyi', 0)}")
print(f"   📊 Metric:  {priority_counts.get('metric', 0)}")

In [ ]:
# Display the daily digest
from pathlib import Path
from IPython.display import Markdown, display
from datetime import datetime

digest_dir = PROJECT_ROOT / "agents" / "digests"
today = datetime.utcnow().strftime("%Y-%m-%d")
digest_path = digest_dir / f"{today}.md"

if digest_path.exists():
    content = digest_path.read_text(encoding="utf-8")
    print(f"📄 Daily Digest: {digest_path.name} ({len(content)} chars)")
    print("─" * 60)
    display(Markdown(content))
else:
    # Show available digests
    digests = sorted(digest_dir.glob("*.md"), reverse=True) if digest_dir.exists() else []
    if digests:
        latest = digests[0]
        content = latest.read_text(encoding="utf-8")
        print(f"📄 Latest Digest: {latest.name}")
        display(Markdown(content))
    else:
        print("(No digest found)")

---
## 13. Full Workforce Summary Dashboard

Aggregated results from all agents in a single view.

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Collect all results
all_results = {
    "engineer":     engineer_result,
    "sentinel":     sentinel_result,
    "advocate":     advocate_result,
    "telemetry":    telemetry_result,
    "productizer":  productizer_result,
    "marketing":    marketing_result,
    "support":      agents["support"].run(question="test"),  # Quick test run
    "presentation": presentation_result,
    "conductor":    conductor_result,
}

# Build summary table
summary_rows = []
for name, result in all_results.items():
    priority_counts = Counter(esc.priority.value for esc in result.escalations)
    summary_rows.append({
        "Agent": name.title(),
        "Status": "✅" if result.success else "❌",
        "Actions": len(result.actions_taken),
        "🔴": priority_counts.get('urgent', 0),
        "🟡": priority_counts.get('review', 0),
        "🟢": priority_counts.get('fyi', 0),
        "📊": priority_counts.get('metric', 0),
        "Duration (s)": round(result.duration_seconds, 2),
    })

df_summary = pd.DataFrame(summary_rows)

# Render as a styled table
fig = go.Figure(data=[go.Table(
    header=dict(
        values=list(df_summary.columns),
        fill_color='#1a237e',
        font=dict(color='white', size=13),
        align='left'
    ),
    cells=dict(
        values=[df_summary[col] for col in df_summary.columns],
        fill_color=[['#e8eaf6' if i % 2 == 0 else 'white' for i in range(len(df_summary))]],
        align='left',
        font=dict(size=12)
    )
)])
fig.update_layout(title='Minerva Agent Workforce — Run Summary', height=420)
fig.show()

# Escalation breakdown pie chart
total_urgent = sum(r.get('🔴', 0) for r in summary_rows)
total_review = sum(r.get('🟡', 0) for r in summary_rows)
total_fyi    = sum(r.get('🟢', 0) for r in summary_rows)
total_metric = sum(r.get('📊', 0) for r in summary_rows)

total_esc = total_urgent + total_review + total_fyi + total_metric
if total_esc > 0:
    fig2 = go.Figure(data=[go.Pie(
        labels=['🔴 Urgent', '🟡 Review', '🟢 FYI', '📊 Metric'],
        values=[total_urgent, total_review, total_fyi, total_metric],
        hole=0.4,
        marker_colors=['#ef5350', '#ffa726', '#66bb6a', '#42a5f5']
    )])
    fig2.update_layout(title=f'Escalation Distribution ({total_esc} total)', height=380)
    fig2.show()

In [ ]:
# Print final summary
print("═" * 60)
print("  MINERVA AGENT WORKFORCE — FINAL SUMMARY")
print("═" * 60)

success_count = sum(1 for r in all_results.values() if r.success)
fail_count = len(all_results) - success_count
total_actions = sum(len(r.actions_taken) for r in all_results.values())
total_escalations = sum(len(r.escalations) for r in all_results.values())

print(f"\n  Agents run:    {len(all_results)}")
print(f"  Succeeded:     {success_count} ✅")
print(f"  Failed:        {fail_count} ❌")
print(f"  Total actions: {total_actions}")
print(f"  Escalations:   {total_escalations}")
print(f"    🔴 Urgent:   {total_urgent}")
print(f"    🟡 Review:   {total_review}")
print(f"    🟢 FYI:      {total_fyi}")
print(f"    📊 Metric:   {total_metric}")
print()
print("  Knowledge Base outputs:")
print("    → agents/knowledge_base/product/vision_gap_analysis.md")
print("    → agents/knowledge_base/product/sales_kits/*.md")
print("    → agents/knowledge_base/operations/telemetry_insights.md")
print("    → agents/digests/*.md  (daily digest)")
print("    → agents/digests/marketing/*.md  (social drafts)")
print("    → agents/digests/telemetry/*.md  (telemetry report)")
print("    → reports/presentations/*.pptx")
print()
print("  Next steps:")
print("    1. Run `minerva dashboard` for the full Streamlit UI")
print("    2. Run `python -m agents run all` to run all agents via CLI")
print("    3. Review the daily digest in agents/digests/")
print("    4. Enable live mode in agents_config.yaml for production")
print()

---
## 14. Agent Configuration Reference

How to configure each agent for production use.

In [ ]:
from IPython.display import Markdown, display

config_doc = """
## Configuration Guide

### `agents/agents_config.yaml`

```yaml
agents:
  conductor:
    enabled: true          # Run the daily orchestration
    schedule: daily
    dry_run: false         # Set to false for LIVE mode
    llm_model: claude-3-5-sonnet

  engineer:
    enabled: true
    schedule: weekly       # Weekly gap analysis
    dry_run: true

  sentinel:
    enabled: true
    schedule: on_event     # Triggered by PR events
    extra:
      min_confidence_score: 7

  marketing:
    enabled: true          # Enable before public launch
    schedule: weekly
    extra:
      days: 7              # Look-back window for git log

  telemetry:
    enabled: true          # Enable to track usage
    schedule: weekly
    extra:
      error_rate_threshold: 0.30
```

### Environment Variables

| Variable | Purpose |
|---|---|
| `ANTHROPIC_API_KEY` | Enable Claude cloud LLM |
| `GITHUB_TOKEN` | Enable Advocate & Sentinel GitHub access |
| `AGENT_DRY_RUN=1` | Force ALL agents into dry-run (safe for testing) |
| `ENABLE_LOCAL_LLM=1` | Use local GGUF model |
| `MCP_ENABLED=1` | Start MCP server for Claude Desktop |

### Running Agents

```bash
# Run all enabled agents
python -m agents run all

# Run a specific agent in dry-run mode
python -m agents run engineer --dry-run

# Trigger Sentinel on a PR
python -m agents run sentinel --pr_number=42

# Generate a sales kit for a topic
python -m agents run productizer --topic=crypto

# Ask the Support agent a question
python -m agents run support --question="How do I configure Ollama?"

# List all agents and their status
python -m agents list
```

### Conductor Scheduling

The Conductor runs daily and triggers:
- **Advocate** → daily GitHub triage
- **Engineer** → weekly gap analysis (Mondays)
- **Telemetry** → weekly usage metrics (Sundays)
- **Marketing** → weekly social drafts

Use `agents/scheduler.py` for cron-based execution or integrate with GitHub Actions.
"""

display(Markdown(config_doc))

In [ ]:
# Show the agents_config.yaml contents
config_path = PROJECT_ROOT / "agents" / "agents_config.yaml"
if config_path.exists():
    import yaml
    with open(config_path) as f:
        config = yaml.safe_load(f)
    
    print("Current agents_config.yaml:")
    print("─" * 60)
    for agent_name, settings in config.get("agents", {}).items():
        enabled = settings.get('enabled', False)
        dry_run = settings.get('dry_run', True)
        schedule = settings.get('schedule', '?')
        mode = "🟢 LIVE" if (enabled and not dry_run) else ("🟡 dry-run" if enabled else "⏸  disabled")
        print(f"  {agent_name:15s} {mode:18s} schedule={schedule}")

---
## 15. MCP Server — Tool Exposure for Claude Desktop

Minerva exposes all its tools via the Model Context Protocol (MCP),
making them available to Claude Desktop and other AI clients.

In [ ]:
# Show MCP server configuration
mcp_config_path = PROJECT_ROOT / "mcp.json"

if mcp_config_path.exists():
    with open(mcp_config_path) as f:
        mcp_config = json.load(f)
    print("mcp.json (Claude Desktop configuration):")
    print(json.dumps(mcp_config, indent=2))

# Show API registry size
try:
    from mcp_server.api_registry import API_REGISTRY
    print(f"\n\n📡 API Registry: {len(API_REGISTRY)} public APIs registered")
    categories = {}
    for api_id, api in API_REGISTRY.items():
        cat = api.get('category', 'Other')
        categories[cat] = categories.get(cat, 0) + 1
    for cat, count in sorted(categories.items(), key=lambda x: -x[1])[:10]:
        bar = '█' * min(count, 20)
        print(f"   {cat:25s} {bar} {count}")
except ImportError as e:
    print(f"\nMCP Server modules: {e}")
    print("Run `python -m mcp_server run` to start the MCP server")

In [ ]:
# Demo: API Discovery
try:
    from mcp_server.discovery_agent import APIDiscoveryAgent
    
    print("═" * 60)
    print("  MCP: API Discovery Demo")
    print("═" * 60)
    
    discoverer = APIDiscoveryAgent()
    
    demo_queries = [
        "US economic GDP data",
        "cryptocurrency prices",
        "weather forecast",
    ]
    
    for query in demo_queries:
        result = discoverer.discover_api(query)
        print(f"\n  Query: '{query}'")
        if isinstance(result, dict):
            print(f"  → Found: {result.get('name', result.get('api_id', 'unknown'))}")
            if result.get('endpoint'):
                print(f"  → Endpoint: {result.get('endpoint')[:60]}")
        elif isinstance(result, list) and result:
            print(f"  → {len(result)} matches found")
            for r in result[:2]:
                if isinstance(r, dict):
                    print(f"     • {r.get('name', r.get('api_id', str(r)[:40]))}")
        else:
            print(f"  → {str(result)[:80]}")

except ImportError as e:
    print(f"API Discovery not available: {e}")

---
## Appendix: Escalation Priority Reference

| Priority | Emoji | Meaning | Response Time |
|----------|-------|---------|---------------|
| URGENT   | 🔴    | Requires human action today | Same day |
| REVIEW   | 🟡    | Needs review within 48 hours | 48 hours |
| FYI      | 🟢    | Handled autonomously, informational | None needed |
| METRIC   | 📊    | Data point for metrics snapshot | None needed |

## Data Flow Between Agents

```
Telemetry ──→ knowledge_base/operations/telemetry_insights.md
    │                    │
    └──────────────────→ Productizer ──→ knowledge_base/product/sales_kits/*.md
                                              │
                              Marketing ←─────┘ (reads sales kits)
                                  │
                                  └──→ digests/marketing/*.md (social drafts)

Engineer ──→ knowledge_base/product/vision_gap_analysis.md

Productizer ──→ knowledge_base/product/sales_kits/*.md
                        │
            Presentation ──→ reports/presentations/*.pptx

Conductor ──→ agents/digests/YYYY-MM-DD.md (daily digest)
```

---
*Generated by Minerva Agent Demo Notebook — run `minerva dashboard` for the full interactive experience.*